In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
import streamlit as st
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
import re

In [5]:
# Tipos de usuarios:

Tipo1="https://www.flexicar.es/combustible-gasolina/carroceria-compacto/segunda-mano/"
Tipo2= "https://www.flexicar.es/combustible-gasolina/carroceria-turismo/segunda-mano/"
Tipo3= "https://www.flexicar.es/combustible-diesel/carroceria-turismo/segunda-mano/"
Tipo4= "https://www.flexicar.es/combustible-diesel/carroceria-familiar/segunda-mano/"
Tipo5= "https://www.flexicar.es/combustible-gasolina/carroceria-familiar/segunda-mano/"
Tipo6= "https://www.flexicar.es/combustible-gasolina/carroceria-monovolumen/segunda-mano/"
Tipo7= "https://www.flexicar.es/combustible-diesel/carroceria-monovolumen/segunda-mano/"

In [6]:
def concat(*args):
        strs = [str(arg) for arg in args if not pd.isnull(arg)]
        return ','.join(strs) if strs else np.nan

In [7]:
def FlexicarScraping(tipo):
    #URL=f'https://www.flexicar.es/coches-segunda-mano/#precio_to/{presupuesto}/'
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("start-maximized")
    options.add_argument("--disable-infobars")
    driver = webdriver.Chrome(executable_path=r'/usr/bin/chromedriver',options=options) 
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    driver.get(tipo)
    precio= driver.find_elements_by_xpath('//*[@id="listadocoches"]')
    cochesBuenos= [e.text for e in precio]
    for e in cochesBuenos:
        ok=e.split("\n")
    droplist=['Previous','Next',"IVA deducible"]
    cleanList=[e for e in ok if e not in droplist]
    CleanList=[]
    for e in cleanList:
        if not "desde" in e:
             CleanList.append(e)
    priceWithDiscount=CleanList[1::4]
    model=CleanList[2::4]
    cleanColumn=CleanList[3::4]
    for e in cleanColumn:
        ok=e.split("\n")
    cleanColumn2=[e.split("\n") for e in cleanColumn]
    ok2=[]
    for e in cleanColumn2:
        for i in e:
            ok2.append(i.split())
    Año=[e[0] for e in ok2]
    Combustible=[e[2] for e in ok2]
    combustiblesucio=[re.findall(r"([$D-H](\w+))",e) for e in Combustible]
    combustiblesucio=[list(item) for sublist in combustiblesucio for item in sublist]
    combustible=[item for sublist in combustiblesucio for item in sublist]
    gas=[]
    for e in combustible:
        if e == 'Hibrido':
            gas.append(e)
        elif e == "Diesel":
            gas.append(e)
        elif e == "Gasolina":
            gas.append(e)
        elif e == "Electrico":
            gas.append(e)
        elif e == "GLP":
            gas.append(e)
    df = pd.DataFrame({
                   'priceWithDiscount': priceWithDiscount,
                     'Model' : model,
                     'Combustible': gas,
                     'Año': Año,
                     'cleanToSplit': cleanColumn
                  })
    df1 = pd.DataFrame(df.cleanToSplit.str.split(' ',4).tolist(),
                                 columns = ["Año1","KM","3","Ciudad","nan"])
    result = pd.concat([df, df1], axis=1, sort=False)
    flexicar = result.drop(["cleanToSplit",'Año1','3'], axis=1)
    np_concat = np.vectorize(concat)
    flexicar['Ciudad'] = np_concat(flexicar['Ciudad'], flexicar['nan'])
    flexicar = flexicar.drop(['nan'], axis=1)
    
    
    return flexicar.sort_values(by='Año', ascending=[True])

In [8]:
Tipo1=FlexicarScraping(Tipo1)
Tipo2=FlexicarScraping(Tipo2)
Tipo3=FlexicarScraping(Tipo3)
Tipo4=FlexicarScraping(Tipo4)
Tipo5=FlexicarScraping(Tipo5)
Tipo6=FlexicarScraping(Tipo6)
Tipo7=FlexicarScraping(Tipo7)

In [9]:
Tipo1


,priceWithDiscount,Model,Combustible,Año,KM,Ciudad
6,4.990€,Toyota iQ iQ 1.0 MultiDrive,Gasolina,2010,151.800,Sabadell
1,4.490€,Citroen C1 VTi 68 Live,Gasolina,2014,110.501,Bilbao
4,5.490€,Ford Ka Urban 1.2 Duratec Auto-Start-Stop,Gasolina,2015,79.999,Vaciamadrid
2,11.390€,Smart Forfour 0.9 66kW (90CV) S/S PASSION,Gasolina,2016,10.072,Vaciamadrid
0,12.990€,Smart Forfour 0.9 66kW (90CV) S/S,Gasolina,2017,32.907,"Rivas,II"
5,12.790€,SEAT Ibiza SC 1.0 EcoTSI 81kW FR Ultimate Edition,Gasolina,2017,54.600,Vaciamadrid
8,9.190€,Fiat 500 1.2 8v 51kW (69CV) Mirror,Gasolina,2017,9990,"Móstoles,2"
3,17.890€,Honda Civic 1.0 I-VTEC TURBO ELEGANCE NAV,Gasolina,2018,28.000,Vaciamadrid
7,11.990€,Smart Forfour 0.9 66kW (90CV) S/S,Gasolina,2018,23.800,"San,Fernando"


In [ ]:
Tipo2


In [ ]:
Tipo3


In [6]:
Tipos= {'Tipo1': "https://www.flexicar.es/combustible-gasolina/carroceria-compacto/segunda-mano/",
        'Tipo2': "https://www.flexicar.es/combustible-gasolina/carroceria-turismo/segunda-mano/",
        'Tipo3': "https://www.flexicar.es/combustible-diesel/carroceria-turismo/segunda-mano/",
        'Tipo4': "https://www.flexicar.es/combustible-diesel/carroceria-familiar/segunda-mano/",
        'Tipo5': "https://www.flexicar.es/combustible-gasolina/carroceria-familiar/segunda-mano/",
        'Tipo6': "https://www.flexicar.es/combustible-gasolina/carroceria-monovolumen/segunda-mano/",
        'Tipo7': "https://www.flexicar.es/combustible-diesel/carroceria-monovolumen/segunda-mano/",
       }

# Argumentos

## Tipo de usuarios:

   ## Usuario Tipo 1: 
   ### Persona que realiza menos de 15.000km al año.
   ### Uso en vías Urbanas 80% y 20% Extra-Urbanas.
   ### Sobretodo por ciudad y muy pocos viajes por carretera al año.
   ### Conviviendo sólo o con un miembro más.
   
  Tipo1="https://www.flexicar.es/combustible-gasolina/carroceria-compacto/segunda-mano/"
  
  ## Usuario Tipo 2: 
   ### Persona que realiza menos de 15.000km al año.
   ### Uso en vías Urbanas 60% y 40% Extra-Urbanas.
   ### Conviviendo sólo o con un miembro más.
   
  Tipo2="https://www.flexicar.es/combustible-gasolina/carroceria-turismo/segunda-mano/"
  
   ## Usuario Tipo 3: 
   ### Persona que realiza más de 15.000km al año.
   ### Uso en vías Extra-Urbanas 80% y 20% Urbanas.
   ### Conviviendo sólo con un miembro más.
  Tipo3= 'https://www.flexicar.es/combustible-diesel/carroceria-turismo/segunda-mano/'
  
   ## Usuario Tipo 4: 
   ### Persona que realiza más de 15.000km al año.
   ### Uso en vías Extra-Urbanas 80% y 20% Urbanas.
   ### Familia con 1 o 2 hijos.
   Tipo4= "https://www.flexicar.es/combustible-diesel/carroceria-familiar/segunda-mano/"
   
   ## Usuario Tipo 5: 
   ### Persona que realiza menos de 15.000km al año.
   ### Uso en vías Urbanas 70% y 30% Extra-Urbanas.
   ### Familia con 1 o 2 hijos.
   Tipo5= "https://www.flexicar.es/combustible-gasolina/carroceria-familiar/segunda-mano/"
   
   ## Usuario Tipo 6: 
   ### Persona que realiza menos de 15.000km al año.
   ### Uso en vías Urbanas 60% y 40% Extra-Urbanas.
   ### Familia con más de 2 hijos.
   Tipo6= "https://www.flexicar.es/combustible-gasolina/carroceria-monovolumen/segunda-mano/"
   
   ## Usuario Tipo 7: 
   ### Persona que realiza más de 15.000km al año.
   ### Uso en vías Extra-Urbanas 80% y 20% Urbanas.
   ### Familia con más de 2 hijos.
   Tipo7= "https://www.flexicar.es/combustible-diesel/carroceria-monovolumen/segunda-mano/"
   